In [2]:
import pandas as pd

texts = pd.read_csv('/Users/veronica/Downloads/RuPreprocessedNoSep.csv')
len(texts)

10403

In [4]:
import numpy as np

def split_df(df):
    if len(df) % 2 != 0:  # Handling `df` with `odd` number of rows
        df = df.iloc[:-1, :]
    df1, df2 =  np.array_split(df, 2)
    return df1, df2

texts1, texts2 = split_df(texts)
len(texts1)

5201

In [5]:
import json

# словарь

with open('/Users/veronica/Desktop/RuWordVectorDict8.json') as json_file:
    dictionary = json.load(json_file)
len(dictionary)

106378

In [7]:
# составление датасета n-грамм из текстов

dataset_ngramm = list()
n = 3
k = 6
j = 0
for index, row in texts1.iterrows():
    if j <= 10:
        text = row[0].split(' ')
        i = 0
        for word in text:
            if i < len(text) - n + 1:
                try:
                    n_gramma = text[i:i+n]
                    n_gramma_vect = []
                    for w in n_gramma:
                        n_gramma_vect += dictionary[w][:k]
                    n_gramma_vect = np.array(n_gramma_vect)
                    dataset_ngramm.append(n_gramma_vect)
                except:
                    pass
            i += 1
    else:
        break
    j += 1
    
len(dataset_ngramm)

100936

In [8]:
with open('/Users/veronica/Desktop/HumanDataset_k6n3_100936.npy', 'wb') as f:
    np.save(f, dataset_ngramm)

In [ ]:
import numpy as np

def create_dataset(corpus: list, k: int, n: int) -> np.array:
    dataset_ngramm = list()
    for text in corpus:
        for sentence in text:
            i = 0
            for word, vector in sentence['sentence_text'].items():
                if i < len(sentence['sentence_text']) - n + 1:
                    try:
                        n_gramma = list(sentence['sentence_text'].items())[i:i+n]
                        n_gramma_vect = np.array(n_gramma[0][1][:k])
                        for j in range(1, n):
                            try:
                                n_gramma_vect = np.concatenate([n_gramma_vect,np.array(n_gramma[j][1][:k])])
                            except:
                                pass
                        dataset_ngramm.append(n_gramma_vect)
                    except:
                        pass
                i += 1
    return np.array(dataset_ngramm, dtype=object)

with open('/home/fibykov/veronica/datasetRU_k6n3.npy', 'wb') as f:
    corpus = np.load('/home/fibykov/veronica/RuVectorizedCorpus8.npy', allow_pickle=True)
    corpus = corpus.tolist()
    dataset = create_dataset(corpus, 6, 3)
    np.save(f, dataset)
    print("vectors in dataset:", len(dataset))
    lens = dict()
    for vector in dataset:
        if len(vector) in lens.keys():
            lens[len(vector)] += 1
        else:
            lens[len(vector)] = 1
    print("dictionary of vector lengths:", lens)
